In [1]:
#
import findspark
findspark.init()
#
import pyspark
from pyspark.sql import functions as pfunc
from pyspark.sql import SQLContext
from pyspark.sql import Window, types
#
import re
import pandas as pd
import numpy as np
from pandas import DataFrame
from pyspark.sql.types import IntegerType
from pyspark.sql.types import FloatType
from pyspark.sql.functions import udf
from pyspark.sql.functions import *
from scipy.stats import kstest
from scipy import stats
#
from pyspark.ml.feature import Tokenizer
from pyspark.ml.feature import RegexTokenizer
#
#import org.apache.spark.ml.feature.NGram
from pyspark.ml.feature import NGram
#
from collections import Counter
#
from pyspark.ml.feature import NGram
#
from pyspark.ml.feature import NGram, CountVectorizer, VectorAssembler
from pyspark.ml import Pipeline
#
from pyspark.mllib.linalg import SparseVector, DenseVector
#
from pyspark.ml.feature import PCA
from pyspark.ml.linalg import Vectors
#
#
sc = pyspark.SparkContext(appName="FraudCanada-Model-NGrams-CountVectorizer-KL-KS-Entropy-Model-CleanData")
sqlContext = SQLContext(sc)
#
# Arguments
#
#import argparse
## Parse date_of execution
#parser = argparse.ArgumentParser()
#parser.add_argument("--datev1", help="Execution Date")
#args = parser.parse_args()
#if args.datev1:
#    processdate = args.datev1
# GENERAL PREPARATION SCRIPT
#
#  Date in format YYYYMMDD
#process_date = processdate
#if not process_date:
#    process_date = "20181231"
#
#
process_date="20190206"
#
input_file1_playback_fraud="hdfs:///data/staged/ott_dazn/fraud-canada-tokenizedwords/dt=*/*.json"
output_file1="hdfs:///data/staged/ott_dazn/advanced-model-data/fraud-notfraud-canada-tokenizedwords-ngrams-7-features-85/dt="+process_date
#
input_file2_playback_not_fraud="hdfs:///data/staged/ott_dazn/logs-archive-production/parquet/dt="+process_date+"/*.parquet"
output_file2="hdfs:///data/staged/ott_dazn/advanced-model-data/not-fraud-canada-tokenizedwords/dt="+process_date
input_file3=output_file2
#
input_file4="hdfs:///data/staged/ott_dazn/advanced-model-data/fraud-notfraud-canada-tokenizedwords-ngrams-7-features-85/dt="+process_date+"/*.*"
#
output_most_frequent_fraud_df="hdfs:///data/staged/ott_dazn/advanced-model-data/the-most-frequent-fraud-hash_message/dt="+process_date
#
output_most_frequent_notfraud_df="hdfs:///data/staged/ott_dazn/advanced-model-data/the-most-frequent-notfraud-hash_message/dt="+process_date
#
#  FILTER Non-Fraud AND LABEL
from pyspark.sql import functions as F
#
#
df2= sqlContext.read.parquet(input_file2_playback_not_fraud)
df2.printSchema()
#
df3 = df2.filter(" (message LIKE '%\"Url\":\"https://isl-ca.dazn.com/misl/v2/Playback%') AND (message LIKE '%,\"Response\":{\"StatusCode\":200,\"ReasonPhrase\":\"OK\",%') AND ( ( (message LIKE '%&Format=MPEG-DASH&%' OR message LIKE '%&Format=M3U&%') ) OR (message NOT LIKE '%\"User-Agent\":\"Mozilla/5.0,(Macintosh; Intel Mac OS X 10_12_6),AppleWebKit/605.1.75,(KHTML, like Gecko),Version/11.1.2,Safari/605.1.75\"},%')   )  ")
df3.printSchema()
df4 = df3.withColumn("messagecut", expr("substring(message, locate('|Livesport.WebApi.Controllers.Playback.PlaybackV2Controller|',message)+60 , length(message)-1)"))
#
# val regexTokenizer = new RegexTokenizer().setInputCol("messagecut").setOutputCol("words").setPattern("\\w+|").setGaps(false)
#
regexTokenizer = RegexTokenizer(minTokenLength=1, gaps=False, pattern='\\w+|', inputCol="messagecut", outputCol="words", toLowercase=True)
#
tokenized = regexTokenizer.transform(df4)
tokenized.printSchema()
tokenized.coalesce(1).write.json(output_file2)
# Tokenize NON-Fraud-LABEL
# hash the message de-duplicate those records
notfraud_file=sqlContext.read.json(input_file3).repartition(500)
notfraud_file.printSchema()
#
notfraud_df=notfraud_file\
.filter("message IS NOT NULL").filter("words IS NOT NULL")\
.withColumn('fraud_label',lit(0).cast('int'))\
.withColumn('hash_message',F.sha2(col('message'),512)).groupby(col('hash_message'))\
.agg(F.first(col('fraud_label')).alias('fraud_label'),F.first(col('words')).alias('words'),F.first(col('message')).alias('message'))\
.persist(pyspark.StorageLevel.MEMORY_AND_DISK_2)
notfraud_df.printSchema()
# Only the Not-Fraud are randomly sorted
#
from pyspark.sql.functions import rand
#
df_notfraud_words = notfraud_df.filter("message IS NOT NULL").select(col('fraud_label'),col('hash_message'),col('words'))\
.orderBy(rand())\
.persist(pyspark.StorageLevel.MEMORY_AND_DISK_2)
df_notfraud_words.printSchema()
#
# FILTER FRAUD AND LABEL 
# Join with Internal Curation Data in urltopredict staged folder
# hash the message de-duplicate those records
fraud_file=sqlContext.read.json(input_file1_playback_fraud).repartition(500)
fraud_file.printSchema()
#
fraud_df=fraud_file\
.filter("message IS NOT NULL").filter("words IS NOT NULL")\
.withColumn('fraud_label',lit(1).cast('int'))\
.withColumn('hash_message',F.sha2(col('message'),512)).groupby(col('hash_message'))\
.agg(F.first(col('fraud_label')).alias('fraud_label'),F.first(col('words')).alias('words'),F.first(col('message')).alias('message'))\
.persist(pyspark.StorageLevel.MEMORY_AND_DISK_2)
fraud_df.printSchema()
#
df_words = fraud_df.filter("message IS NOT NULL").select(col('fraud_label'),col('hash_message'),col('words'))\
.persist(pyspark.StorageLevel.MEMORY_AND_DISK_2)
df_words.printSchema()
#
# Limit to 250,000 Daily Not-Fraud Records input in the nGrams Graph analysis
#
result_fraud_nofraud_words = df_words.union(df_notfraud_words).limit(150000)\
.persist(pyspark.StorageLevel.MEMORY_AND_DISK_SER)
## Register Generic Functions
# -----------------------------------------------------------------------------
# Build ngrams 75 90 n=6 
# support : https://stackoverflow.com/questions/51473703/pyspark-ml-ngrams-countvectorizer-sorted-based-on-count-weights
# -----------------------------------------------------------------------------
def build_ngrams_part(inputCol="words", n=6):
    ngrams = [ 
        NGram(n=i, inputCol="words", outputCol="ngrams_{0}".format(i)) 
        for i in range(7, n + 1) ]
    vectorizers = [ 
        CountVectorizer(inputCol="ngrams_{0}".format(i), outputCol="ngramscounts_{0}".format(i)) 
        for i in range(7, n + 1) ]
    return Pipeline(stages=ngrams + vectorizers)
#    assembler = [VectorAssembler( inputCols=["ngramscounts_{0}".format(i) for i in range(1, n + 1)], outputCol="features" )]
#    return Pipeline(stages=ngrams + DenseVector(SparseVector(vectorizers).toArray()))
#
# 
# -----------------------------------------------------------------------------
#ngram = build_ngrams_part().fit(df_words)
#ngramDataFrame = ngram.transform(df_words)
#ngramDataFrame.coalesce(1).write.json(output_file1)
#
ngram = NGram(n=7, inputCol="words", outputCol="ngrams_7")
countvector = CountVectorizer(inputCol="ngrams_7", outputCol="ngramscounts_7")
# fit a CountVectorizerModel from the corpus.
countvModel = CountVectorizer(inputCol="words", outputCol="features_85", vocabSize=85, minDF=2.0)
# fit a PCA Dimensionality reduction into 7/3=2.x components from ngramscounts_7 ## Too Heavy 1st PCA
pcaNgrams = PCA(k=3, inputCol="ngramscounts_7", outputCol="pcaweightngrams")
# fit a PCA Dimensionality reduction into 85/17=5 components from words
pcaWords = PCA(k=5, inputCol="features_85", outputCol="pcaweightwords")  ## Too Heavy 2nd PCA
#
ngram_fraud_DF = ngram.transform(result_fraud_nofraud_words)
ngram_vc_fraud_DF = countvector.fit(ngram_fraud_DF).transform(ngram_fraud_DF)\
.persist(pyspark.StorageLevel.MEMORY_AND_DISK_2)
ngram_vc_fraud_DF.printSchema()
#
# Trial of PCA now with Less Data
#modelPCA_ngram_fraud_DF = pcaNgrams.fit(ngram_vc_fraud_DF).transform(ngram_vc_fraud_DF)\
#.persist(pyspark.StorageLevel.MEMORY_AND_DISK_2)
#modelPCA_ngram_fraud_DF.printSchema()
##
############# ISSUE - NGRAMS ARE TOO BIG ############
#Traceback (most recent call last):
#  File "/opt/cloudera/parcels/SPARK2-2.3.0.cloudera2-1.cdh5.13.3.p0.316101/lib/spark2/python/pyspark/sql/utils.py", line 63, in deco
#    return f(*a, **kw)
#  File "/opt/cloudera/parcels/SPARK2-2.3.0.cloudera2-1.cdh5.13.3.p0.316101/lib/spark2/python/lib/py4j-0.10.6-src.zip/py4j/protocol.py", line 320, in get_return_value
#py4j.protocol.Py4JJavaError: An error occurred while calling o200.fit.
#: java.lang.IllegalArgumentException: Argument with more than 65535 cols: 262144
#	at org.apache.spark.mllib.linalg.distributed.RowMatrix.checkNumColumns(RowMatrix.scala:133)
#	at org.apache.spark.mllib.linalg.distributed.RowMatrix.computeCovariance(RowMatrix.scala:332)
#	at org.apache.spark.mllib.linalg.distributed.RowMatrix.computePrincipalComponentsAndExplainedVariance(RowMatrix.scala:387)
#	at org.apache.spark.mllib.feature.PCA.fit(PCA.scala:53)
#	at org.apache.spark.ml.feature.PCA.fit(PCA.scala:99)
#	at sun.reflect.NativeMethodAccessorImpl.invoke0(Native Method)
#	at sun.reflect.NativeMethodAccessorImpl.invoke(NativeMethodAccessorImpl.java:62)
#	at sun.reflect.DelegatingMethodAccessorImpl.invoke(DelegatingMethodAccessorImpl.java:43)
#	at java.lang.reflect.Method.invoke(Method.java:498)
#	at py4j.reflection.MethodInvoker.invoke(MethodInvoker.java:244)
#	at py4j.reflection.ReflectionEngine.invoke(ReflectionEngine.java:357)
#	at py4j.Gateway.invoke(Gateway.java:282)
#	at py4j.commands.AbstractCommand.invokeMethod(AbstractCommand.java:132)
#	at py4j.commands.CallCommand.execute(CallCommand.java:79)
#	at py4j.GatewayConnection.run(GatewayConnection.java:214)
#	at java.lang.Thread.run(Thread.java:748)
#
#
#During handling of the above exception, another exception occurred:
#
#Traceback (most recent call last):
#  File "/home/siemanalyst/projects/logs-archive-production/fraud-canada-tokenizedwords/notebooks/x1-FraudCanada-Model-NGrams-CountVectorizer-KL-KS-Entropy-Model-CleanData.py", line 173, in <module>
#####################################################
#
#result_ngrams_words_fraud_DF = countvModel.fit(modelPCA_ngram_fraud_DF).transform(modelPCA_ngram_fraud_DF)\
result_ngrams_words_fraud_DF = countvModel.fit(ngram_vc_fraud_DF).transform(ngram_vc_fraud_DF)\
.persist(pyspark.StorageLevel.MEMORY_AND_DISK_2)
result_ngrams_words_fraud_DF.printSchema()
#
#modelPCA_features_ngram_fraud_DF = pcaWords.fit(result_ngrams_words_fraud_DF).transform(result_ngrams_words_fraud_DF)\
#.persist(pyspark.StorageLevel.MEMORY_AND_DISK_2)
#modelPCA_features_ngram_fraud_DF.printSchema()
#
#modelPCA_features_ngram_fraud_DF.coalesce(1).write.json(output_file1)
result_ngrams_words_fraud_DF.coalesce(1).write.json(output_file1)
#
print("Calculation of most frequent fraud_ngram notfraud_ngram - Start!")
#
#  CALCULATE KL,KS COEF. Label Data fraud/not_fraud
###################
# Obtain the most frequent word on each position 
# Compose the standard_fraud_ngram from the most common positions
# Calculate the standard_fraud_ngram
#
# https://stackoverflow.com/questions/35218882/find-maximum-row-per-group-in-spark-dataframe 
# Using struct ordering:
#from pyspark.sql.functions import struct
#
#(cnts
#  .groupBy("id_sa")
#  .agg(F.max(struct(col("cnt"), col("id_sb"))).alias("max"))
#  .select(col("id_sa"), col("max.id_sb")))
#
####################
# FRAUD
ngram7_fraud=sqlContext.read.json(input_file4).filter("fraud_label=1")
ngram7_fraud.printSchema()
#
most_frequent_fraud_df=ngram7_fraud\
.withColumn("value_sum",F.explode("ngramscounts_7.values"))\
.groupBy("hash_message").agg(F.sum("value_sum").alias('count'))\
.persist(pyspark.StorageLevel.MEMORY_AND_DISK_2)
most_frequent_fraud_df.printSchema()
#
most_frequent_fraud_df.coalesce(1).write.json(output_most_frequent_fraud_df)
#
# The most Frequent would the the max
standard_fraud_ngram=most_frequent_fraud_df.orderBy(col('count').desc()).select(col('hash_message')).limit(1).toPandas()
#
print("Value Print: standard_fraud_ngram=")
print(standard_fraud_ngram)
#
####################
# NOT FRAUD
ngram7_notfraud=sqlContext.read.json(input_file4).filter("fraud_label=0")
ngram7_notfraud.printSchema()
#
most_frequent_notfraud_df=ngram7_notfraud\
.withColumn("value_sum",F.explode("ngramscounts_7.values"))\
.groupBy("hash_message").agg(F.sum("value_sum").alias('count'))\
.persist(pyspark.StorageLevel.MEMORY_AND_DISK_2)
most_frequent_notfraud_df.printSchema()
#
most_frequent_notfraud_df.coalesce(1).write.json(output_most_frequent_notfraud_df)
#
# The most Frequent would the the max
standard_notfraud_ngram=most_frequent_notfraud_df.orderBy(col('count').desc()).select(col('hash_message')).limit(1).toPandas()
#
print("Value Print: standard_notfraud_ngram=")
print(standard_notfraud_ngram)
#
print("Calculation of most frequent fraud_ngram notfraud_ngram - Done!")
#
sc.stop()
#
print("Preparation of Data Done!")
#

root
 |-- metadata: struct (nullable = true)
 |    |-- beat: string (nullable = true)
 |    |-- type: string (nullable = true)
 |-- logzio_id: string (nullable = true)
 |-- awsRegion: string (nullable = true)
 |-- beat: struct (nullable = true)
 |    |-- hostname: string (nullable = true)
 |    |-- name: string (nullable = true)
 |    |-- version: string (nullable = true)
 |-- correlationId: string (nullable = true)
 |-- function_version: string (nullable = true)
 |-- host: struct (nullable = true)
 |    |-- name: string (nullable = true)
 |-- hostname: string (nullable = true)
 |-- id: string (nullable = true)
 |-- input_type: string (nullable = true)
 |-- invoked_function_arn: string (nullable = true)
 |-- it: string (nullable = true)
 |-- level: long (nullable = true)
 |-- logGroup: string (nullable = true)
 |-- logStream: string (nullable = true)
 |-- logzio_codec: string (nullable = true)
 |-- message: string (nullable = true)
 |-- messageType: string (nullable = true)
 |-- msg: s

root
 |-- features_85: struct (nullable = true)
 |    |-- indices: array (nullable = true)
 |    |    |-- element: long (containsNull = true)
 |    |-- size: long (nullable = true)
 |    |-- type: long (nullable = true)
 |    |-- values: array (nullable = true)
 |    |    |-- element: double (containsNull = true)
 |-- fraud_label: long (nullable = true)
 |-- hash_message: string (nullable = true)
 |-- ngrams_7: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- ngramscounts_7: struct (nullable = true)
 |    |-- indices: array (nullable = true)
 |    |    |-- element: long (containsNull = true)
 |    |-- size: long (nullable = true)
 |    |-- type: long (nullable = true)
 |    |-- values: array (nullable = true)
 |    |    |-- element: double (containsNull = true)
 |-- words: array (nullable = true)
 |    |-- element: string (containsNull = true)

root
 |-- hash_message: string (nullable = true)
 |-- count: double (nullable = true)

Value Print: standard_fraud_

In [4]:
sc.stop()

In [8]:
#
import findspark
findspark.init()
#
import pyspark
from pyspark.sql import functions as pfunc
from pyspark.sql import SQLContext
from pyspark.sql import Window, types
#
import re
import pandas as pd
import numpy as np
from pandas import DataFrame
from pyspark.sql.types import IntegerType
from pyspark.sql.types import FloatType
from pyspark.sql.functions import udf
from pyspark.sql.functions import *
from scipy.stats import kstest
from scipy import stats
#
from pyspark.ml.feature import Tokenizer
from pyspark.ml.feature import RegexTokenizer
#
#import org.apache.spark.ml.feature.NGram
from pyspark.ml.feature import NGram
#
from collections import Counter
#
from pyspark.ml.feature import NGram
#
from pyspark.ml.feature import NGram, CountVectorizer, VectorAssembler
from pyspark.ml import Pipeline
#
from pyspark.mllib.linalg import SparseVector, DenseVector
#
from pyspark.ml.feature import PCA
from pyspark.ml.linalg import Vectors
#
#  FILTER with PySpark SQL Functions F.
from pyspark.sql import functions as F
# FUNCTIONS
# 
##
## Register Generic UDF Functions
# -----------------------------------------------------------------------------
# KL : Kullback-Leibler Divergence
# KS : Kolmogorov-Smirnov ( Sample sizes can be different)
# -----------------------------------------------------------------------------    
## NGRAM hash_mesage KL from URL TO Default NGRAM KL
def func_kl_ngram_msg(var1,var_match):
    ##Making sure the analysis of has_message SHA_512 is linear.
    ##Making sure the analysis is over the words list on same size
    def KL(P,Q):
        epsilon = 0.00001
        P = P+epsilon
        Q = Q+epsilon
        divergence = np.sum(P*np.log(P/Q))
        return np.asscalar(divergence)
    ##Making sure the analysis ignore trash
    cleanvar=var1.strip('http://').strip('https://').strip('www.')
    len_var1=len(cleanvar)
    idx_var1=cleanvar[:4]# Use "root1" as baseline #cleanvar.split("/")[0]#cleanvar[:8]
    list_values1 = list(cleanvar)
    list_of_ord_values1 = [ord(char) for char in list_values1]
    values1 = np.asarray(list_of_ord_values1)
    # URLS with at least 4166 characters
    ## TODO : Complete this list with common Malware/phishing sites used in internal webtraffic
    list_values2 = var_match
    list_values2 += var_match
    ## Making sure the analysis start on the same webdomain name
    idx_var2 = list_values2.find(idx_var1)
    start_values2 = list_values2[idx_var2:(idx_var2+len_var1)]
    list_values2 = list(start_values2+list_values2)
    list_values2 = list_values2[:len_var1]
    list_of_ord_values2 = [ord(char) for char in list_values2]
    values2 = np.asarray(list_of_ord_values2)
    return KL(values1,values2)
# -----------------------------------------------------------------------------
# -----------------------------------------------------------------------------
## NGRAM hash_message "Kolmogorov-Smirnov test KS pvalue= ks_2samp function8.
def func_ks_ngram_msg(var1,var_match):
    ##Making sure the analysis of has_message SHA_512 is linear.
    ##Making sure the analysis is over the words list on same size
    cleanvar=var1.strip('http://').strip('https://').strip('www.')
    len_var1=len(cleanvar)
    idx_var1=cleanvar[:4]# Use "root1" as baseline #cleanvar.split("/")[0]#cleanvar[:8]
    list_values1 = list(cleanvar)
    list_of_ord_values1 = [ord(char) for char in list_values1]
    values1 = np.asarray(list_of_ord_values1)
    values1= np.sort(values1)
    # URLS with up to least 4166 characters
    list_values2 = var_match
    list_values2 += var_match
    ## Making sure the analysis start on the same messgae
    idx_var2 = list_values2.find(idx_var1)
    start_values2 = list_values2[idx_var2:(idx_var2+len_var1)]
    list_values2 = list(start_values2+list_values2)
    list_values2 = list_values2[:len_var1]
    list_of_ord_values2 = [ord(char) for char in list_values2]
    values2 = np.asarray(list_of_ord_values2)
    values2=np.sort(values2)
    (Darray,pvalue)=stats.ks_2samp(values1, values2)
    return np.asscalar(pvalue)
# -----------------------------------------------------------------------------
# -----------------------------------------------------------------------------
## Entropy TO hash_message function3.
def func_entropy_ngram_msg(var1,var_match):
    ##Making sure the analysis of has_message SHA_512 is linear.
    ##Making sure the analysis is over the words list ignore size
    #
    cleanvar=var1.strip('http://').strip('https://').strip('www.')
    len_var1=len(cleanvar)
    idx_var1=cleanvar[:4]# Use "root1" as baseline #cleanvar.split("/")[0]#cleanvar[:8]
    list_values1 = list(cleanvar)
    list_of_ord_values1 = [ord(char) for char in list_values1]
    values1 = np.asarray(list_of_ord_values1)
    # URLS with at least 4166 characters
    ## TODO : Complete this list with an NGRAM search
    list_values2 = var_match
    list_values2 += var_match
    ## Making sure the analysis start on point
    idx_var2 = list_values2.find(idx_var1)
    start_values2 = list_values2[idx_var2:(idx_var2+len_var1)]
    list_values2 = list(start_values2+list_values2)
    list_values2 = list_values2[:len_var1]
    list_of_ord_values2 = [ord(char) for char in list_values2]
    values2 = np.asarray(list_of_ord_values2)
    pvalue=stats.entropy(values1, values2)
    scalar_pvalue=np.asscalar(pvalue)
    return scalar_pvalue
# -----------------------------------------------------------------------------
# -----------------------------------------------------------------------------
#
# -----------------------------------------------------------------------------
# -----------------------------------------------------------------------------
func_kl_ngram_msg_udf = udf(func_kl_ngram_msg, FloatType())
func_ks_ngram_msg_udf = udf(func_ks_ngram_msg, FloatType())
func_entropy_ngram_msg_udf = udf(func_entropy_ngram_msg, FloatType())
#
# Arguments
#
#import argparse
## Parse date_of execution
#parser = argparse.ArgumentParser()
#parser.add_argument("--datev1", help="Execution Date")
#args = parser.parse_args()
#if args.datev1:
#    processdate = args.datev1
# GENERAL PREPARATION SCRIPT
#
#  Date in format YYYYMMDD
#process_date = processdate
#if not process_date:
#    process_date = "20190122"
#
process_date = "20190206"
#
sc = pyspark.SparkContext(appName="FraudCanada-Model-NGrams-CountVectorizer-KL-KS-Entropy-Model-Labeling")
sqlContext = SQLContext(sc)
#
#
input_most_frequent_fraud_df="hdfs:///data/staged/ott_dazn/advanced-model-data/the-most-frequent-fraud-hash_message/dt="+process_date
#
input_most_frequent_notfraud_df="hdfs:///data/staged/ott_dazn/advanced-model-data/the-most-frequent-notfraud-hash_message/dt="+process_date
#
input_fraud="hdfs:///data/staged/ott_dazn/advanced-model-data/fraud-notfraud-canada-tokenizedwords-ngrams-7-features-85/dt="+process_date
#
input_file1="hdfs:///data/staged/ott_dazn/advanced-model-data/fraud-notfraud-canada-tokenizedwords-ngrams-7-features-85/dt="+process_date
#
output_file1="hdfs:///data/staged/ott_dazn/advanced-model-data/label-fraud-notfraud-data-model/dt="+process_date
#
preserve_training_output_file="hdfs:///data/staged/ott_dazn/advanced-model-data/preserve-training-output-automl-clean/dt="+process_date
#
# The most Frequent would the ones with the max frequency of NGrams85 tokens
pd.options.display.max_colwidth = 512
#
standard_fraud_ngram=sqlContext.read.json(input_most_frequent_fraud_df)\
.orderBy(col('count').desc()).select(col('hash_message')).limit(1).toPandas()['hash_message'][0]
print("Value UDF : standard_fraud_ngram=")
print(standard_fraud_ngram)
#
# Select Tokens/words form the max frequency of NGrams85 tokens hash_message
standard_fraud_ngram_words=sqlContext.read.json(input_fraud)
standard_fraud_ngram_words.printSchema()
#
standard_fraud_words_search=standard_fraud_ngram_words\
.withColumn('fraud_master_hash',lit(standard_fraud_ngram).cast('string'))\
.filter(" hash_message=fraud_master_hash ")
standard_fraud_words_search.printSchema()
#
standard_fraud_words=standard_fraud_words_search\
.withColumn('words_conc',F.concat_ws('',col('words')).cast('string'))\
.select(col('words_conc')).limit(1).toPandas()['words_conc'][0] 
#
print("Value UDF : standard_fraud_words=")
print(standard_fraud_words)
#
# Working now!
# Needs to be bigger than message twice should be tested!
#   Py4JJavaError: An error occurred while calling z:org.apache.spark.sql.functions.lit.
#      : java.lang.RuntimeException: Unsupported literal type class java.util.HashMap 
# {0=root15c466c8e6e8f9d17adb73426cd55c70f72b9f18e39e3455c9043a18b86b122b6requestmethodgeturlhttpsislcadazncommislv2playbackassetidfg5oon8sl71n1nfwuegbo8npgeventidarticleidfg5oon8sl71n1nfwuegbo8npgformatmpegdashplayeriddaznf3874e050812a853securetruelanguagecodeenlatitudenulllongitudenullplatformandroidtvmanufacturernvidiamodelnullmtalanguagecodeenclientip50100225179headersuseragentmozilla50linuxandroid800shieldandroidtvbuildopr6170623010wvapplewebkit53736khtmllikegeckoversion40chrome710357899mobilesafari53736fev1420typeinresponsestatuscode200reasonphraseokduration47jwtvieweridc2ebc25d8085deviceid993bf365c72c4b0b9168c2ebc25d8085f3874e050812a853userstatusactivepaid}
#
standard_notfraud_ngram=sqlContext.read.json(input_most_frequent_notfraud_df)\
.orderBy(col('count').desc()).select(col('hash_message')).limit(1).toPandas()['hash_message'][0]
print("Value UDF : standard_notfraud_ngram=")
print(standard_notfraud_ngram)
#
# Select Tokens/words form the max frequency of NGrams85 tokens hash_message
standard_notfraud_ngram_words=sqlContext.read.json(input_fraud)
standard_notfraud_ngram_words.printSchema()
#
standard_notfraud_words_search=standard_notfraud_ngram_words\
.withColumn('fraud_master_hash',lit(standard_notfraud_ngram).cast('string'))\
.filter(" hash_message=fraud_master_hash ")
standard_notfraud_words_search.printSchema()
#
standard_notfraud_words=standard_notfraud_words_search\
.withColumn('words_conc',F.concat_ws('',col('words')).cast('string'))\
.select(col('words_conc')).limit(1).toPandas()['words_conc'][0] 
#
print("Value UDF : standard_notfraud_words=")
print(standard_notfraud_words)
#
ngram7_fraud=sqlContext.read.json(input_file1)
ngram7_fraud.printSchema()
#
drop_phish_cols=['words','ngrams_7']
#
fraud_label_read_df=ngram7_fraud.filter("hash_message is not NULL")\
.withColumn('words_conc',F.concat_ws('',col('words')).cast('string'))\
.drop(*drop_phish_cols)\
.withColumn('kl_fraud_words',func_kl_ngram_msg_udf(col('words_conc'),lit(standard_fraud_words).cast('string')).cast('double'))\
.withColumn('ks_fraud_words',func_ks_ngram_msg_udf(col('words_conc'),lit(standard_fraud_words).cast('string')).cast('double'))\
.withColumn('entropy_fraud_words',func_entropy_ngram_msg_udf(col('words_conc'),lit(standard_fraud_words).cast('string')).cast('double'))\
.withColumn('kl_notfraud_words',func_kl_ngram_msg_udf(col('words_conc'),lit(standard_notfraud_words).cast('string')).cast('double'))\
.withColumn('ks_notfraud_words',func_ks_ngram_msg_udf(col('words_conc'),lit(standard_notfraud_words).cast('string')).cast('double'))\
.withColumn('entropy_notfraud_words',func_entropy_ngram_msg_udf(col('words_conc'),lit(standard_notfraud_words).cast('string')).cast('double'))\
.persist(pyspark.StorageLevel.MEMORY_AND_DISK_2)
fraud_label_read_df.printSchema()
#
fraud_label_read_df.coalesce(1).write.json(output_file1)
#
fraud_label_read_df.unpersist()
#
sc.stop()
#
print("Model Data KL,KS, Entropy Done! NGrams Vectors Data Done!")
#

Value UDF : standard_fraud_ngram=
d71ec5039b409ec52b112cdf682cf2f4554329bc88207360175c3dd2d3ba04771d7e828aec45badeed2451c7dff5349bd17acf9045ea29d01ac02445d41e2387
root
 |-- features_85: struct (nullable = true)
 |    |-- indices: array (nullable = true)
 |    |    |-- element: long (containsNull = true)
 |    |-- size: long (nullable = true)
 |    |-- type: long (nullable = true)
 |    |-- values: array (nullable = true)
 |    |    |-- element: double (containsNull = true)
 |-- fraud_label: long (nullable = true)
 |-- hash_message: string (nullable = true)
 |-- ngrams_7: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- ngramscounts_7: struct (nullable = true)
 |    |-- indices: array (nullable = true)
 |    |    |-- element: long (containsNull = true)
 |    |-- size: long (nullable = true)
 |    |-- type: long (nullable = true)
 |    |-- values: array (nullable = true)
 |    |    |-- element: double (containsNull = true)
 |-- words: array (nullable = true)
 

In [7]:
sc.stop()

In [9]:
#
import findspark
findspark.init()
#
import pyspark
from pyspark.sql import functions as pfunc
from pyspark.sql import SQLContext
from pyspark.sql import Window, types
#
import re
import pandas as pd
import numpy as np
from pandas import DataFrame
from pyspark.sql.types import IntegerType
from pyspark.sql.types import FloatType
from pyspark.sql.functions import udf
from pyspark.sql.functions import *
from scipy.stats import kstest
from scipy import stats
#
from pyspark.ml.feature import Tokenizer
from pyspark.ml.feature import RegexTokenizer
#
#import org.apache.spark.ml.feature.NGram
from pyspark.ml.feature import NGram
#
from collections import Counter
#
from pyspark.ml.feature import NGram
#
from pyspark.ml.feature import NGram, CountVectorizer, VectorAssembler
from pyspark.ml import Pipeline
#
from pyspark.mllib.linalg import SparseVector, DenseVector
#
from pyspark.ml.feature import PCA
from pyspark.ml.linalg import Vectors
#
#  FILTER with PySpark SQL Functions F.
#
# Arguments
#
import argparse
## Parse date_of execution
#parser = argparse.ArgumentParser()
#parser.add_argument("--datev1", help="Execution Date")
#args = parser.parse_args()
#if args.datev1:
#    processdate = args.datev1
# GENERAL PREPARATION SCRIPT
#
#  Date in format YYYYMMDD
#process_date = processdate
#if not process_date:
#    process_date = "20181231"
#
process_date="20190206"
#
sc = pyspark.SparkContext(appName="FraudCanada-AUTOML-Model-NGrams-CountVectorizer-KL-KS-Entropy")
sqlContext = SQLContext(sc)
#
input_most_frequent_df="hdfs:///data/staged/ott_dazn/advanced-model-data/the-most-frequent-fraud-hash_message/dt="+process_date
input_fraud="hdfs:///data/staged/ott_dazn/advanced-model-data/fraud-notfraud-canada-tokenizedwords-ngrams-5-features-85/dt="+process_date
#
input_file1="hdfs:///data/staged/ott_dazn/advanced-model-data/fraud-notfraud-canada-tokenizedwords-ngrams-5-features-85/dt="+process_date
output_file1="hdfs:///data/staged/ott_dazn/advanced-model-data/label-fraud-notfraud-data-model/dt="+process_date
preserve_training_input_file="hdfs:///data/staged/ott_dazn/advanced-model-data/preserve-training-output-automl-clean/dt="+process_date
#
import h2o
from h2o.automl import H2OAutoML
#
import subprocess
subprocess.run('unset http_proxy', shell=True)
#
# Start an H2O virtual cluster that uses 6 gigs of RAM and 6 cores
h2o.init(ip="localhost",port=54321,max_mem_size = "6g", nthreads = 6) 
#
# Clean up the h2o cluster just in case
h2o.remove_all()
#
#  TRAINING PROCESS
#
print("Start Training Model NGrams Vectors KS KL Entropty")
#
# Horrible code :: close your eyes, is ugly
#
fraud_label_read_file=sqlContext.read.json(output_file1)
fraud_label_read_file.printSchema()
#
fraud_label_read_df=fraud_label_read_file\
.select(col('hash_message').cast('string'),col('fraud_label').cast('int'),\
        col('kl_fraud_words').cast('double'),col('ks_fraud_words').cast('double'),\
        col('entropy_fraud_words').cast('double'),\
        col('kl_notfraud_words').cast('double'), col('ks_notfraud_words').cast('double'),\
        col('entropy_notfraud_words').cast('double'),\
        col('features_85.type').alias('features85_type').cast('long'),\
        col('features_85.size').alias('features85_size').cast('long'),\
        col('features_85.indices').alias('features85_indices'),\
        col('features_85.values').alias('features85_values'),\
        col('ngramscounts_7.type').alias('ngramscounts7_type').cast('long'),\
        col('ngramscounts_7.size').alias('ngramscounts7_size').cast('long'),\
        col('ngramscounts_7.indices').alias('ngramscounts7_indices'),\
        col('ngramscounts_7.values').alias('ngramscounts7_values'))
fraud_label_read_df.printSchema()
#
# ABOVE ARE CASE ISSUES on struct Struct of features_85 and ngramscounts_7 
# Both cares conversion to DF valide type list
# Flat vars for each, individually and seperately from the original struct
#
# https://stackoverflow.com/questions/47401418/pyspark-conversion-to-array-types?rq=1 
#
#
fraud_fraud_label_read1_df=fraud_label_read_df.filter("fraud_label=1")\
.persist(pyspark.StorageLevel.MEMORY_AND_DISK_2)
notfraud_fraud_label_read1_df=fraud_label_read_df.filter("fraud_label=0")\
.persist(pyspark.StorageLevel.MEMORY_AND_DISK_2)
#
fraud_fraud_label_read1_df.printSchema()
notfraud_fraud_label_read1_df.printSchema()
#
drop_list_cols=['features85_indices','features85_values','ngramscounts7_indices','ngramscounts7_values']
#
### 1.) https://stackoverflow.com/questions/38610559/convert-spark-dataframe-column-to-python-list
###    list(spark_df.select('mvv').toPandas()['mvv'])
### 2.) http://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.assign.html
###    df.assign(temp_f=lambda x: x['temp_c'] * 9 / 5 + 32,temp_k=lambda x: (x['temp_f'] +  459.67) * 5 / 9)
### 3.) https://stackoverflow.com/questions/43216411/pandas-flatten-a-list-of-list-within-a-column
###    df['var2'] = df['var2'].apply(np.ravel)
### 4.) Random xxx rows
###    df.orderBy(rand()).limit(n)
from pyspark.sql.functions import rand
#
fraud_label_train_pd_rand=fraud_fraud_label_read1_df.limit(10000)\
.orderBy(rand()).persist(pyspark.StorageLevel.MEMORY_AND_DISK_SER)
#
fraud_label_train_pd=fraud_label_train_pd_rand.limit(3100).toPandas()\
.assign(features85_list_indices=lambda x: x['features85_indices'].apply(np.ravel),\
        features85_list_values=lambda x: x['features85_values'].apply(np.ravel),\
        ngramscounts7_list_indices=lambda x: x['ngramscounts7_indices'].apply(np.ravel),\
        ngramscounts7_list_values=lambda x: x['ngramscounts7_values'].apply(np.ravel))\
.drop(drop_list_cols, axis=1, inplace=False)
#
fraud_label_test_pd_rand=fraud_fraud_label_read1_df.limit(10000)\
.orderBy(rand()).persist(pyspark.StorageLevel.MEMORY_AND_DISK_SER)
#
fraud_label_test_pd=fraud_label_test_pd_rand.limit(500).toPandas()\
.assign(features85_list_indices=lambda x: x['features85_indices'].apply(np.ravel),\
        features85_list_values=lambda x: x['features85_values'].apply(np.ravel),\
        ngramscounts7_list_indices=lambda x: x['ngramscounts7_indices'].apply(np.ravel),\
        ngramscounts7_list_values=lambda x: x['ngramscounts7_values'].apply(np.ravel))\
.drop(drop_list_cols, axis=1, inplace=False)
#
fraud_label_train=h2o.H2OFrame(fraud_label_train_pd)
fraud_label_test=h2o.H2OFrame(fraud_label_test_pd)
#
not_fraud_label_train_pd_rand=notfraud_fraud_label_read1_df.limit(10000)\
.orderBy(rand()).persist(pyspark.StorageLevel.MEMORY_AND_DISK_SER)
#
not_fraud_label_train_pd=not_fraud_label_train_pd_rand.limit(3100).toPandas()\
.assign(features85_list_indices=lambda x: x['features85_indices'].apply(np.ravel),\
        features85_list_values=lambda x: x['features85_values'].apply(np.ravel),\
        ngramscounts7_list_indices=lambda x: x['ngramscounts7_indices'].apply(np.ravel),\
        ngramscounts7_list_values=lambda x: x['ngramscounts7_values'].apply(np.ravel))\
.drop(drop_list_cols, axis=1, inplace=False)
#
not_fraud_label_test_pd_rand=notfraud_fraud_label_read1_df.limit(10000)\
.orderBy(rand()).persist(pyspark.StorageLevel.MEMORY_AND_DISK_SER)
#
not_fraud_label_test_pd=not_fraud_label_test_pd_rand.limit(800).toPandas()\
.assign(features85_list_indices=lambda x: x['features85_indices'].apply(np.ravel),\
        features85_list_values=lambda x: x['features85_values'].apply(np.ravel),\
        ngramscounts7_list_indices=lambda x: x['ngramscounts7_indices'].apply(np.ravel),\
        ngramscounts7_list_values=lambda x: x['ngramscounts7_values'].apply(np.ravel))\
.drop(drop_list_cols, axis=1, inplace=False)
#.orderBy(rand())\
#.sort(notfraud_fraud_label_read1_df.kl_notfraud_words.desc())\
#
not_fraud_label_train=h2o.H2OFrame(not_fraud_label_train_pd)
not_fraud_label_test=h2o.H2OFrame(not_fraud_label_test_pd)
#
################# Use Two DataFrames ##################### - rbind() H2o Frames issue
#
#
###### TRAINING PROCESS ############
# RBIND "Merge" all of vars internal subset of data with fraud and with not_fraud
# function merge() doesn't work if both H2O/dataframes have same variables
#
train = fraud_label_train.rbind(not_fraud_label_train)
test = fraud_label_test.rbind(not_fraud_label_test)
#
print("train")
print(train.head(10))
print("test")
print(test.head(10))
#
# Identify predictors and response
x = train.columns
#
# Fraud Label to be learned in the model from the atrributes of the ngram85 learned words
#
y= 'fraud_label'
x.remove(y)
#
# For binary classification, response should be a factor
train[y] = train[y].asfactor()
test[y] = test[y].asfactor()
#
# http://docs.h2o.ai/h2o/latest-stable/h2o-docs/automl.html
# Balance Classes to compensate unbalanced data
# Run AutoML for 25 base models (limited to 1 hour max runtime by default)
aml = H2OAutoML(max_models=30, seed=1999, exclude_algos=["DRF","GLM"])
aml.train(x=x, y=y, training_frame=train)
#
#preserve_training_output.write.json(preserve_training_output_file)
#
print("AutoML Modeling Done!")
#
# View the AutoML Leaderboard
lb = aml.leaderboard
lb.head(rows=lb.nrows)  # Print all rows instead of default (10 rows)
#
# The leader model is stored here
aml.leader
#
# Get model ids for all models in the AutoML Leaderboard
model_ids = list(aml.leaderboard['model_id'].as_data_frame().iloc[:,0])
print(model_ids)
# Get the "All Models" Stacked Ensemble model
se = h2o.get_model([mid for mid in model_ids if "StackedEnsemble_AllModels" in mid][0])
print(se)
# Get the Stacked Ensemble metalearner model
#metalearner = h2o.get_model(aml.leader.metalearner()['name'])
#metalearner.coef_norm()
#%matplotlib inline
#metalearner.std_coef_plot()
# If you need to generate predictions on a test set, you can make
# predictions directly on the `"H2OAutoML"` object, or on the leader
# model object directly

#preds = aml.predict(test)
# or:
preds = aml.leader.predict(test)
print("test")
print(test.head(10))
print("prediction")
print(preds.head(10))
#
#
print("Save Model For Future Usage")
aml.leader.download_mojo(path = "./projects/logs-archive-production/fraud-canada-tokenizedwords/notebooks/product_model_bin/ngrams7_features85_m30/v"+process_date+"/mojo", get_genmodel_jar = True)
# If you need to generate predictions on a test set, you can make
# predictions directly on the `"H2OAutoML"` object, or on the leader
# model object directly

#preds = aml.predict(test)
# or:
preds = aml.leader.predict(test)
print("test")
print(test.tail(10))
print("prediction")
print(preds.tail(10))
# If you need to generate predictions on a test set, you can make
# predictions directly on the `"H2OAutoML"` object, or on the leader
# model object directly

#preds = aml.predict(test)
# or:
preds_over_all_hf = aml.leader.predict(train)
path_out_file1="./projects/logs-archive-production/fraud-canada-tokenizedwords/notebooks/product_model_prediction/file_prediction_"+process_date+".csv"
output_pred_file=h2o.export_file(frame=preds_over_all_hf, path=path_out_file1, force=False)
#
train_over_all_hf = train
path_out_file2="./projects/logs-archive-production/fraud-canada-tokenizedwords/notebooks/product_model_prediction/file_train_"+process_date+".csv"
output_pred_file=h2o.export_file(frame=train_over_all_hf, path=path_out_file2, force=False)
#
sc.stop()
#

Checking whether there is an H2O instance running at http://localhost:54321..... not found.
Attempting to start a local H2O server...
  Java Version: java version "1.8.0_172"; Java(TM) SE Runtime Environment (build 1.8.0_172-b11); Java HotSpot(TM) 64-Bit Server VM (build 25.172-b11, mixed mode)
  Starting server from /opt/cloudera/parcels/Anaconda-5.1.0.1/lib/python3.6/site-packages/h2o/backend/bin/h2o.jar
  Ice root: /tmp/tmp9s14p_qx
  JVM stdout: /tmp/tmp9s14p_qx/h2o_siemanalyst_started_from_python.out
  JVM stderr: /tmp/tmp9s14p_qx/h2o_siemanalyst_started_from_python.err
  Server is running at http://127.0.0.1:54321
Connecting to H2O server at http://127.0.0.1:54321... successful.


H2O cluster uptime:,01 secs
H2O cluster timezone:,UTC
H2O data parsing timezone:,UTC
H2O cluster version:,3.22.0.1
H2O cluster version age:,3 months and 10 days !!!
H2O cluster name:,H2O_from_python_siemanalyst_mw3hay
H2O cluster total nodes:,1
H2O cluster free memory:,5.333 Gb
H2O cluster total cores:,12
H2O cluster allowed cores:,6
H2O cluster status:,"accepting new members, healthy"


Start Training Model NGrams Vectors KS KL Entropty
root
 |-- entropy_fraud_words: double (nullable = true)
 |-- entropy_notfraud_words: double (nullable = true)
 |-- features_85: struct (nullable = true)
 |    |-- indices: array (nullable = true)
 |    |    |-- element: long (containsNull = true)
 |    |-- size: long (nullable = true)
 |    |-- type: long (nullable = true)
 |    |-- values: array (nullable = true)
 |    |    |-- element: double (containsNull = true)
 |-- fraud_label: long (nullable = true)
 |-- hash_message: string (nullable = true)
 |-- kl_fraud_words: double (nullable = true)
 |-- kl_notfraud_words: double (nullable = true)
 |-- ks_fraud_words: double (nullable = true)
 |-- ks_notfraud_words: double (nullable = true)
 |-- ngramscounts_7: struct (nullable = true)
 |    |-- indices: array (nullable = true)
 |    |    |-- element: long (containsNull = true)
 |    |-- size: long (nullable = true)
 |    |-- type: long (nullable = true)
 |    |-- values: array (nullable = 

hash_message,fraud_label,kl_fraud_words,ks_fraud_words,entropy_fraud_words,kl_notfraud_words,ks_notfraud_words,entropy_notfraud_words,features85_type,features85_size,ngramscounts7_type,ngramscounts7_size,features85_list_indices,features85_list_values,ngramscounts7_list_indices,ngramscounts7_list_values
45c8664c2595ec4eb69d2794510a6b2709cd4c7376424543ff7d1a856a9fd264dcf2acf79c229d1735ece2b0293a26e1f7f06977b09d503ee3ecbf79f41fcfe1,1,2958.18,0.680909,0.0414306,4174.65,0.576074,0.0702108,0,85,0,262144,[ 0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 31 32 33 34 35 38 39 40 41 42 43 44 45 46 47 48 49 50 51 52 53 56 57 62 63 65 66 77 82],[1. 2. 3. 4. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 2.],[ 0 1 2 3 4 5 6 7 8 9 10 11 13 14 15 48 49 50 51 52 90 91 92 93 94 95 115 124 125 136 137 138 139 140 141 142 143 144 145 146 147 148 155 156 157 158 159 160 161 162 164 165 166 295 544 545 546 1940 1956 1961 1962 3995 25796 28975 42401 54583 55521 57339 59314 60189 62079 64022 70148 98667 102744 104261 107736 108682 112715 156710 209253 213413],[1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
51fd93b4b44009b433bc5a23f288031224ea54e53126895967ce64385c261256352efd51bc8d8daabe1662b039b8c4777d86e9c5a792efabbbc3c86c3abce0cd,1,2494.88,0.990999,0.0387373,3931.17,0.830355,0.0720945,0,85,0,262144,[ 0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 31 32 33 34 35 38 39 40 41 42 43 44 45 46 47 48 49 50 51 52 53 56 57 62 63 65 66 77 82],[1. 2. 3. 4. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 2.],[ 0 1 2 3 4 5 6 7 8 9 10 11 13 14 15 48 49 50 51 52 90 91 92 93 94 95 115 124 125 136 137 138 139 140 141 142 143 144 145 146 147 148 155 156 157 158 159 160 161 162 707 842 843 844 3094 10390 10429 10458 10548 11708 21959 29074 68837 71482 78795 86039 90805 91804 92815 93940 95051 95231 100051 107944 112029 112432 135667 154706 198073 243058],[1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
a718ac829139f11b98d9480c89f8bdb0d245093ca42fb35383f5c863168388745996497a2739b53d376d929dd0cc1c2381c398c088f0c958614514c07ecc3050,1,3084.28,0.831228,0.0472219,3671.83,0.629505,0.0636913,0,85,0,262144,[ 0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 31 32 33 34 35 38 39 40 41 42 43 44 45 46 47 48 49 50 51 52 53 56 57 62 63 65 66 77 82],[1. 2. 3. 4. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 2.],[ 0 1 2 3 4 5 6 7 8 9 10 11 13 14 15 48 49 50 51 52 90 91 92 93 94 95 115 124 125 136 137 138 139 140 141 142 143 144 145 146 147 148 155 156 157 158 159 160 161 162 164 165 166 295 707 2561 3630 3724 3753 14203 14267 15388 26219 26988 31068 47005 48996 49224 49470 52427 60632 61270 76692 83576 91789 93571 96977 104828 113156 251060],[1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
022f6b81a53eb05b037dbb0cad11d846daa9d5d35982585da98f1499f01eaddd442a465c91b3eba97cfa45ef374da9a0644f4b0808669ac92bd7f095a17d9872,1,3156.26,0.784173,0.0474292,3850.26,0.682076,0.0660141,0,85,0,262144,[ 0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 


test


...

hash_message,fraud_label,kl_fraud_words,ks_fraud_words,entropy_fraud_words,kl_notfraud_words,ks_notfraud_words,entropy_notfraud_words,features85_type,features85_size,ngramscounts7_type,ngramscounts7_size,features85_list_indices,features85_list_values,ngramscounts7_list_indices,ngramscounts7_list_values
0c983268c70fd7c560320a3a925c414fd448cfad5dbc1287f1f1234aae6b706bdc5ac0c1a43d67efea4f8d4d1c47a4795449ad8284a1d4eef377f2e407ecca15,0,3787.34,0.399077,0.0795858,2450.78,0.722469,0.0547815,0,85,0,262144,[ 0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 31 32 33 34 38 39 40 41 42 43 44 45 46 47 48 49 50 51 52 53 54 55 57 58 59 60 64 74],[3. 2. 2. 2. 2. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 2. 1. 1. 1. 1. 1. 1.],[ 0 1 2 3 4 5 6 7 8 9 23 24 25 26 27 36 45 46 47 65 66 67 68 69 70 71 72 73 74 75 76 77 78 96 97 98 99 100 101 102 103 104 105 106 112 113 114 152 153 154 1455 2641 12207 36645 37149 38512 42844 43290 45175 45622 47181 47315 47754 51188 51421 53447 54992 55174 57563 58819 58824 58831 59045 59285 59421 60606 61074 64791 65510 66339 67109 67692 126259 227408 246021],[1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
751b0d4b88a1ad630b258c03596bf591c7d7d671fe8a54fc802599b80695ca85373e5b918307956930479bfcc5640b2fa91ccadf77c8afcf75dbbeef8d627689,0,3136.01,0.522336,0.0757026,2208.72,0.316835,0.0628293,0,85,0,262144,[ 0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 31 32 33 34 38 39 40 41 42 43 44 45 46 47 48 49 50 51 52 53 54 57 59 60 64 74],[3. 2. 2. 2. 2. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.],[ 0 1 2 3 4 5 6 7 8 9 36 45 46 47 68 69 70 71 72 73 74 75 76 77 78 96 97 98 99 100 101 102 103 104 105 106 112 113 114 1070 1101 1103 1116 1172 1587 1853 1864 1866 7423 7552 7969 23657 46798 59908 60463 61747 64797 70099 70310 78128 79052 81104 81253 87607 88791 90640 91462 92107 96001 97569 98487 100447 106278 107445 111550 141147 141207 155918 163518 198047 237709 239794 259091],[1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
cbc22468fac4860040971cbe031ea19f8227f966d3a27884a9466a08b869d926f8ce43bf9ba88ee4691958b5a30794f0039b68fdc45f99d3dfc515727c2118d7,0,3183.96,0.522336,0.0760335,2247.62,0.569637,0.0630055,0,85,0,262144,[ 0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 31 32 33 34 38 39 40 41 42 43 44 45 46 47 48 49 50 51 52 53 54 59 60 64 74],[3. 2. 2. 2. 2. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.],[ 0 1 2 3 4 5 6 7 8 9 36 45 46 47 68 69 70 71 72 73 74 75 76 77 78 96 97 98 99 100 101 102 103 104 105 106 112 113 114 302 303 304 928 944 948 952 1624 1625 1626 1657 2654 16970 20440 23125 23858 24125 24348 33711 34189 34569 34699 35707 35765 36001 37044 37939 38432 39438 39543 41952 42951 43753 44569 44846 45642 103813 104899 174276 189763 191089 219811 223729],[1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
647a1a86d4a231f6ea53f9d03cc4065768bac9e94533264d52a474613af4af4f6efa50f598c09acf943be604b22714ee7b5d6cfe5e8d05a2044a5650ca247c07,0,5527.58,0.177085,0.0803395,5264.12,0.116322,0.0787753,0,85,0,262144,[ 0 1 2 3 


prediction


predict,p0,p1
0,1,1e-19
0,1,1e-19
0,1,1e-19
0,1,1e-19
0,1,1e-19
0,1,1e-19
0,1,1e-19
0,1,1e-19
0,1,1e-19
0,1,3.25879e-18



gbm prediction progress: |████████████████████████████████████████████████| 100%
Export File progress: | (failed)


OSError: Job with key $03017f00000132d4ffffffff$_b77a6ef0518b8070fc08c93ab8344ca2 failed with an exception: java.lang.RuntimeException: java.io.FileNotFoundException: ./projects/logs-archive-production/fraud-canada-tokenizedwords/notebooks/product_model_prediction/file_prediction_20190206.csv (No such file or directory)
stacktrace: 
java.lang.RuntimeException: java.io.FileNotFoundException: ./projects/logs-archive-production/fraud-canada-tokenizedwords/notebooks/product_model_prediction/file_prediction_20190206.csv (No such file or directory)
	at water.persist.PersistManager.create(PersistManager.java:586)
	at water.util.FrameUtils$ExportTaskDriver.exportCSVStream(FrameUtils.java:460)
	at water.util.FrameUtils$ExportTaskDriver.compute2(FrameUtils.java:362)
	at water.H2O$H2OCountedCompleter.compute(H2O.java:1310)
	at jsr166y.CountedCompleter.exec(CountedCompleter.java:468)
	at jsr166y.ForkJoinTask.doExec(ForkJoinTask.java:263)
	at jsr166y.ForkJoinPool$WorkQueue.runTask(ForkJoinPool.java

In [10]:
sc.stop()

In [11]:
# View the AutoML Leaderboard
lb = aml.leaderboard
lb.head(rows=lb.nrows)  # Print all rows instead of default (10 rows)

model_id,auc,logloss,mean_per_class_error,rmse,mse
GBM_grid_1_AutoML_20190206_120744_model_2,1,0.00206106,0,0.0155499,0.0002418
GBM_grid_1_AutoML_20190206_120744_model_8,1,0.00222091,0,0.0155514,0.000241847
GBM_grid_1_AutoML_20190206_120744_model_7,1,0.00201646,0,0.0177354,0.000314545
GBM_grid_1_AutoML_20190206_120744_model_9,1,0.00117409,0,0.0152584,0.00023282
GBM_grid_1_AutoML_20190206_120744_model_1,1,0.000805172,0,0.0148946,0.000221848
GBM_1_AutoML_20190206_120744,1,0.00230928,0,0.0200727,0.000402911
StackedEnsemble_AllModels_AutoML_20190206_120744,1,0.000899946,0,0.0149675,0.000224026
DeepLearning_grid_1_AutoML_20190206_120744_model_2,1,0.000246943,0,0.0064663,4.1813e-05
GBM_grid_1_AutoML_20190206_120744_model_6,1,0.00108807,0,0.0148655,0.000220985
StackedEnsemble_BestOfFamily_AutoML_20190206_120744,1,0.000510137,0,0.00936266,8.76594e-05


In [12]:
# The leader model is stored here
aml.leader

Model Details
H2OGradientBoostingEstimator :  Gradient Boosting Machine
Model Key:  GBM_grid_1_AutoML_20190206_120744_model_2


ModelMetricsBinomial: gbm
** Reported on train data. **

MSE: 0.0
RMSE: 0.0
LogLoss: 0.0
Mean Per-Class Error: 0.0
AUC: 1.0
pr_auc: 0.0
Gini: 1.0
Confusion Matrix (Act/Pred) for max f1 @ threshold = 1.0: 


,0,1,Error,Rate
0,2808.0,0.0,0.0,(0.0/2808.0)
1,0.0,1326.0,0.0,(0.0/1326.0)
Total,2808.0,1326.0,0.0,(0.0/4134.0)


Maximum Metrics: Maximum metrics at their respective thresholds



metric,threshold,value,idx
max f1,1.0,1.0,0.0
max f2,1.0,1.0,0.0
max f0point5,1.0,1.0,0.0
max accuracy,1.0,1.0,0.0
max precision,1.0,1.0,0.0
max recall,1.0,1.0,0.0
max specificity,1.0,1.0,0.0
max absolute_mcc,1.0,1.0,0.0
max min_per_class_accuracy,1.0,1.0,0.0
max mean_per_class_accuracy,1.0,1.0,0.0


Gains/Lift Table: Avg response rate: 32.08 %, avg score: 32.08 %



...

,mean,sd,cv_1_valid,cv_2_valid,cv_3_valid,cv_4_valid,cv_5_valid
accuracy,1.0,0.0,1.0,1.0,1.0,1.0,1.0
auc,1.0,0.0,1.0,1.0,1.0,1.0,1.0
err,0.0,0.0,0.0,0.0,0.0,0.0,0.0
err_count,0.0,0.0,0.0,0.0,0.0,0.0,0.0
f0point5,1.0,0.0,1.0,1.0,1.0,1.0,1.0
f1,1.0,0.0,1.0,1.0,1.0,1.0,1.0
f2,1.0,0.0,1.0,1.0,1.0,1.0,1.0
lift_top_group,3.1176536,0.0032218,3.1090226,3.1207547,3.1207547,3.1207547,3.116981
logloss,0.0020606,0.0029141,0.0000000,0.0103028,0.0000000,0.0000000,0.0000000
max_per_class_error,0.0,0.0,0.0,0.0,0.0,0.0,0.0


Scoring History: 


,timestamp,duration,number_of_trees,training_rmse,training_logloss,training_auc,training_pr_auc,training_lift,training_classification_error,validation_rmse,validation_logloss,validation_auc,validation_pr_auc,validation_lift,validation_classification_error
,2019-02-06 12:10:01,1 min 11.906 sec,0.0,0.4667667,0.6274370,0.5,0.0,1.0,0.6792453,0.4704085,0.6345803,0.5,0.0,1.0,0.6697248
,2019-02-06 12:10:01,1 min 11.924 sec,5.0,0.0051481,0.0051366,1.0,0.0007541,3.1176471,0.0,0.0051382,0.0051386,1.0,0.0,3.0277778,0.0
,2019-02-06 12:10:01,1 min 11.947 sec,10.0,0.0000994,0.0000960,1.0,0.9962293,3.1176471,0.0,0.0001229,0.0001007,1.0,0.9722222,3.0277778,0.0
,2019-02-06 12:10:01,1 min 11.993 sec,15.0,0.0000021,0.0000018,1.0,0.9947210,3.1176471,0.0,0.0000032,0.0000020,1.0,0.0347222,3.0277778,0.0
,2019-02-06 12:10:01,1 min 12.040 sec,20.0,0.0000001,0.0000000,1.0,0.0316742,3.1176471,0.0,0.0000034,0.0000002,1.0,0.0277778,3.0277778,0.0
,2019-02-06 12:10:01,1 min 12.098 sec,25.0,0.0000000,0.0000000,1.0,0.0,3.1176471,0.0,0.0000001,0.0000000,1.0,0.0208333,3.0277778,0.0
,2019-02-06 12:10:01,1 min 12.155 sec,30.0,0.0000000,0.0000000,1.0,0.0,3.1176471,0.0,0.0000000,0.0000000,1.0,0.0069444,3.0277778,0.0
,2019-02-06 12:10:01,1 min 12.209 sec,35.0,0.0000000,0.0000000,1.0,0.0,3.1176471,0.0,0.0000000,0.0000000,1.0,0.0,3.0277778,0.0
,2019-02-06 12:10:01,1 min 12.262 sec,40.0,0.0000000,0.0000000,1.0,0.0,3.1176471,0.0,0.0000000,0.0000000,1.0,0.0,3.0277778,0.0
,2019-02-06 12:10:01,1 min 12.306 sec,45.0,0.0000000,0.0000000,1.0,0.0,3.1176471,0.0,0.0000000,0.0000000,1.0,0.0,3.0277778,0.0


Variable Importances: 


variable,relative_importance,scaled_importance,percentage
features85_list_indices,900.5152588,1.0,0.9144560
features85_list_values,84.0486832,0.0933340,0.0853498
ngramscounts7_list_values,0.1673868,0.0001859,0.0001700
entropy_fraud_words,0.0234560,0.0000260,0.0000238
kl_notfraud_words,0.0003148,0.0000003,0.0000003
kl_fraud_words,0.0000837,0.0000001,0.0000001
ks_fraud_words,0.0000086,0.0000000,0.0000000
entropy_notfraud_words,0.0000024,0.0000000,0.0000000
ks_notfraud_words,0.0000011,0.0000000,0.0000000


In [13]:
# If you need to generate predictions on a test set, you can make
# predictions directly on the `"H2OAutoML"` object, or on the leader
# model object directly

#preds = aml.predict(test)
# or:
preds = aml.leader.predict(test)
print("test")
print(test.head(10))
print("prediction")
print(preds.head(10))

gbm prediction progress: |████████████████████████████████████████████████| 100%


/opt/cloudera/parcels/Anaconda-5.1.0.1/lib/python3.6/site-packages/h2o/job.py:69: UserWarning: Test/Validation dataset column 'features85_list_indices' has levels not trained on: [[ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23
 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47
 48 49 50 51 52 53 54 55 56 57 58 61 62 63 65 66 79 80 83 84], [ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23
 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47
 48 49 50 51 52 53 54 55 56 58 61 62 63 65 79 80 83 84], [ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23
 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47
 48 49 50 51 52 53 54 56 60 61 64 66 67 75 77], [ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23
 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47
 48 49 50 51 52 54 55 56 57 58 61 62 63 65 66 79 80], [ 0  1  2  3  

test


hash_message,fraud_label,kl_fraud_words,ks_fraud_words,entropy_fraud_words,kl_notfraud_words,ks_notfraud_words,entropy_notfraud_words,features85_type,features85_size,ngramscounts7_type,ngramscounts7_size,features85_list_indices,features85_list_values,ngramscounts7_list_indices,ngramscounts7_list_values
101642031f7aa7d62d25fbc9dc190f6165b1efd53d20eea9a7405968fc32ba6da3db74ea520bd5910bc52db1acc8f3bb069e415572a344186ab52fb0aa0211d2,1,2703.02,0.83296,0.0480958,3235.29,0.736194,0.0631322,0,85,0,262144,[ 0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 31 32 33 34 35 38 39 40 41 42 43 44 45 46 47 48 49 50 51 52 53 56 57 62 63 65 66 77 82],[1. 2. 3. 4. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 2.],[ 0 1 2 3 4 5 6 7 8 9 10 11 13 14 15 48 49 50 51 52 90 91 92 93 94 95 115 124 125 136 137 138 139 140 141 142 143 144 145 146 147 148 155 156 157 158 159 160 161 162 1192 1193 1195 1972 3168 4048 20261 21615 22667 23930 24166 51800 53721 56262 60340 61638 63487 65066 71064 77803 80020 81356 82272 82280 86026 89190 165716 170455 187454 235691 255607],[1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
a7250368bd2f3d9e0ecbdd03354d7196b83a2a40013c748face8f747e0443fdf0fd6c22a588d1795c16b455c7039cf657ecf5bcdfde08fbe54669045033d24f1,1,3327.77,0.963607,0.0489203,4471.18,0.730739,0.0765263,0,85,0,262144,[ 0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 31 32 33 34 35 38 39 40 41 42 43 44 45 46 47 48 49 50 51 52 53 56 57 62 63 65 66 77 82],[1. 2. 3. 4. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 2.],[ 0 1 2 3 4 5 6 7 8 9 10 11 13 14 15 48 49 50 51 52 90 91 92 93 94 95 115 124 125 136 137 138 139 140 141 142 143 144 145 146 147 148 155 156 157 158 159 160 161 162 842 843 844 2815 6052 6740 6792 6985 12328 12413 22038 23607 25439 27477 29093 31782 32434 32787 33325 70710 71694 78589 80537 87234 104781 105971 106868 108578 112657 211966 217414 242542],[1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
cf6aaf9aa72e0f8ebdbbab8db1e5e6c2d9ffac5bc954ae0e057099e0dd6464ee2136042556445063e81c1a4107c2d0a27234da61e648541391b2a904456a005a,1,2434.66,0.980881,0.0417387,3292.28,0.784173,0.06374,0,85,0,262144,[ 0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 31 32 33 34 35 38 39 40 41 42 43 44 45 46 47 48 49 50 51 52 53 56 57 62 63 65 66 77 82],[1. 2. 3. 4. 1. 1. 1. 2. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 2.],[ 0 1 2 3 4 5 6 7 8 9 10 11 13 14 15 48 49 50 51 52 90 91 92 93 94 95 115 124 125 136 137 138 139 140 141 142 143 144 145 146 147 148 155 156 157 158 159 160 161 162 164 165 166 295 681 1296 1298 1299 1616 1868 3712 3774 3788 3797 3811 3818 3827 3828 3829 3843 17132 18494 36389 36416 36515 38711 41234 45850 46554 120634 124787 225931 242249],[1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
5971b987ced86878931381142fdc479e723acb6637f637cb30156e81f88f06d53cd0f9bbf7180b9d710380c699c05f6d58ab1c23e640e231d803d11dfd7d1cd1,1,2424.23,0.964228,0.0339223,4167.72,0.732933,0.0733494,0,85,0,262144,[ 0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16


prediction


predict,p0,p1
1,0,1
1,0,1
1,0,1
1,0,1
1,0,1
1,0,1
1,0,1
1,0,1
1,0,1
1,0,1


In [14]:
# Get model ids for all models in the AutoML Leaderboard
model_ids = list(aml.leaderboard['model_id'].as_data_frame().iloc[:,0])

print(model_ids)
# Get the "All Models" Stacked Ensemble model
se = h2o.get_model([mid for mid in model_ids if "GBM" in mid][0])
print(se)

# Get the Stacked Ensemble metalearner model
metalearner = h2o.get_model(aml.leader.metalearner()['name'])
metalearner.coef_norm()

['GBM_grid_1_AutoML_20190206_120744_model_2', 'GBM_grid_1_AutoML_20190206_120744_model_8', 'GBM_grid_1_AutoML_20190206_120744_model_7', 'GBM_grid_1_AutoML_20190206_120744_model_9', 'GBM_grid_1_AutoML_20190206_120744_model_1', 'GBM_1_AutoML_20190206_120744', 'StackedEnsemble_AllModels_AutoML_20190206_120744', 'DeepLearning_grid_1_AutoML_20190206_120744_model_2', 'GBM_grid_1_AutoML_20190206_120744_model_6', 'StackedEnsemble_BestOfFamily_AutoML_20190206_120744', 'GBM_grid_1_AutoML_20190206_120744_model_10', 'GBM_grid_1_AutoML_20190206_120744_model_5', 'GBM_4_AutoML_20190206_120744', 'GBM_2_AutoML_20190206_120744', 'DeepLearning_1_AutoML_20190206_120744', 'GBM_3_AutoML_20190206_120744', 'GBM_grid_1_AutoML_20190206_120744_model_3', 'GBM_5_AutoML_20190206_120744', 'DeepLearning_grid_1_AutoML_20190206_120744_model_10', 'DeepLearning_grid_1_AutoML_20190206_120744_model_3', 'DeepLearning_grid_1_AutoML_20190206_120744_model_5', 'DeepLearning_grid_1_AutoML_20190206_120744_model_7', 'DeepLearning_

,0,1,Error,Rate
0,2808.0,0.0,0.0,(0.0/2808.0)
1,0.0,1326.0,0.0,(0.0/1326.0)
Total,2808.0,1326.0,0.0,(0.0/4134.0)


Maximum Metrics: Maximum metrics at their respective thresholds



metric,threshold,value,idx
max f1,1.0,1.0,0.0
max f2,1.0,1.0,0.0
max f0point5,1.0,1.0,0.0
max accuracy,1.0,1.0,0.0
max precision,1.0,1.0,0.0
max recall,1.0,1.0,0.0
max specificity,1.0,1.0,0.0
max absolute_mcc,1.0,1.0,0.0
max min_per_class_accuracy,1.0,1.0,0.0
max mean_per_class_accuracy,1.0,1.0,0.0


Gains/Lift Table: Avg response rate: 32.08 %, avg score: 32.08 %



...

Scoring History: 


,timestamp,duration,number_of_trees,training_rmse,training_logloss,training_auc,training_pr_auc,training_lift,training_classification_error,validation_rmse,validation_logloss,validation_auc,validation_pr_auc,validation_lift,validation_classification_error
,2019-02-06 12:10:01,1 min 11.906 sec,0.0,0.4667667,0.6274370,0.5,0.0,1.0,0.6792453,0.4704085,0.6345803,0.5,0.0,1.0,0.6697248
,2019-02-06 12:10:01,1 min 11.924 sec,5.0,0.0051481,0.0051366,1.0,0.0007541,3.1176471,0.0,0.0051382,0.0051386,1.0,0.0,3.0277778,0.0
,2019-02-06 12:10:01,1 min 11.947 sec,10.0,0.0000994,0.0000960,1.0,0.9962293,3.1176471,0.0,0.0001229,0.0001007,1.0,0.9722222,3.0277778,0.0
,2019-02-06 12:10:01,1 min 11.993 sec,15.0,0.0000021,0.0000018,1.0,0.9947210,3.1176471,0.0,0.0000032,0.0000020,1.0,0.0347222,3.0277778,0.0
,2019-02-06 12:10:01,1 min 12.040 sec,20.0,0.0000001,0.0000000,1.0,0.0316742,3.1176471,0.0,0.0000034,0.0000002,1.0,0.0277778,3.0277778,0.0
,2019-02-06 12:10:01,1 min 12.098 sec,25.0,0.0000000,0.0000000,1.0,0.0,3.1176471,0.0,0.0000001,0.0000000,1.0,0.0208333,3.0277778,0.0
,2019-02-06 12:10:01,1 min 12.155 sec,30.0,0.0000000,0.0000000,1.0,0.0,3.1176471,0.0,0.0000000,0.0000000,1.0,0.0069444,3.0277778,0.0
,2019-02-06 12:10:01,1 min 12.209 sec,35.0,0.0000000,0.0000000,1.0,0.0,3.1176471,0.0,0.0000000,0.0000000,1.0,0.0,3.0277778,0.0
,2019-02-06 12:10:01,1 min 12.262 sec,40.0,0.0000000,0.0000000,1.0,0.0,3.1176471,0.0,0.0000000,0.0000000,1.0,0.0,3.0277778,0.0
,2019-02-06 12:10:01,1 min 12.306 sec,45.0,0.0000000,0.0000000,1.0,0.0,3.1176471,0.0,0.0000000,0.0000000,1.0,0.0,3.0277778,0.0


Variable Importances: 


variable,relative_importance,scaled_importance,percentage
features85_list_indices,900.5152588,1.0,0.9144560
features85_list_values,84.0486832,0.0933340,0.0853498
ngramscounts7_list_values,0.1673868,0.0001859,0.0001700
entropy_fraud_words,0.0234560,0.0000260,0.0000238
kl_notfraud_words,0.0003148,0.0000003,0.0000003
kl_fraud_words,0.0000837,0.0000001,0.0000001
ks_fraud_words,0.0000086,0.0000000,0.0000000
entropy_notfraud_words,0.0000024,0.0000000,0.0000000
ks_notfraud_words,0.0000011,0.0000000,0.0000000


AttributeError: type object 'H2OGradientBoostingEstimator' has no attribute 'metalearner'

In [51]:
%matplotlib inline
metalearner.std_coef_plot()

NameError: name 'metalearner' is not defined

In [15]:
aml.leader.download_mojo(path = "./product_model_bin/ngrams7_features85_vectors_m25/v8/mojo", get_genmodel_jar = True)

'/home/siemanalyst/projects/logs-archive-production/fraud-canada-tokenizedwords/notebooks/product_model_bin/ngrams7_features85_vectors_m25/v8/mojo/GBM_grid_1_AutoML_20190206_120744_model_2.zip'

In [25]:
sc.stop()

In [16]:
# If you need to generate predictions on a test set, you can make
# predictions directly on the `"H2OAutoML"` object, or on the leader
# model object directly

#preds = aml.predict(test)
# or:
preds = aml.leader.predict(test)
print("test")
print(test.tail(10))
print("prediction")
print(preds.tail(10))

gbm prediction progress: |████████████████████████████████████████████████| 100%


/opt/cloudera/parcels/Anaconda-5.1.0.1/lib/python3.6/site-packages/h2o/job.py:69: UserWarning: Test/Validation dataset column 'features85_list_indices' has levels not trained on: [[ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23
 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47
 48 49 50 51 52 53 54 55 56 57 58 61 62 63 65 66 79 80 83 84], [ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23
 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47
 48 49 50 51 52 53 54 55 56 58 61 62 63 65 79 80 83 84], [ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23
 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47
 48 49 50 51 52 53 54 56 60 61 64 66 67 75 77], [ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23
 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47
 48 49 50 51 52 54 55 56 57 58 61 62 63 65 66 79 80], [ 0  1  2  3  

test


hash_message,fraud_label,kl_fraud_words,ks_fraud_words,entropy_fraud_words,kl_notfraud_words,ks_notfraud_words,entropy_notfraud_words,features85_type,features85_size,ngramscounts7_type,ngramscounts7_size,features85_list_indices,features85_list_values,ngramscounts7_list_indices,ngramscounts7_list_values
0c983268c70fd7c560320a3a925c414fd448cfad5dbc1287f1f1234aae6b706bdc5ac0c1a43d67efea4f8d4d1c47a4795449ad8284a1d4eef377f2e407ecca15,0,3787.34,0.399077,0.0795858,2450.78,0.722469,0.0547815,0,85,0,262144,[ 0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 31 32 33 34 38 39 40 41 42 43 44 45 46 47 48 49 50 51 52 53 54 55 57 58 59 60 64 74],[3. 2. 2. 2. 2. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 2. 1. 1. 1. 1. 1. 1.],[ 0 1 2 3 4 5 6 7 8 9 23 24 25 26 27 36 45 46 47 65 66 67 68 69 70 71 72 73 74 75 76 77 78 96 97 98 99 100 101 102 103 104 105 106 112 113 114 152 153 154 1455 2641 12207 36645 37149 38512 42844 43290 45175 45622 47181 47315 47754 51188 51421 53447 54992 55174 57563 58819 58824 58831 59045 59285 59421 60606 61074 64791 65510 66339 67109 67692 126259 227408 246021],[1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
751b0d4b88a1ad630b258c03596bf591c7d7d671fe8a54fc802599b80695ca85373e5b918307956930479bfcc5640b2fa91ccadf77c8afcf75dbbeef8d627689,0,3136.01,0.522336,0.0757026,2208.72,0.316835,0.0628293,0,85,0,262144,[ 0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 31 32 33 34 38 39 40 41 42 43 44 45 46 47 48 49 50 51 52 53 54 57 59 60 64 74],[3. 2. 2. 2. 2. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.],[ 0 1 2 3 4 5 6 7 8 9 36 45 46 47 68 69 70 71 72 73 74 75 76 77 78 96 97 98 99 100 101 102 103 104 105 106 112 113 114 1070 1101 1103 1116 1172 1587 1853 1864 1866 7423 7552 7969 23657 46798 59908 60463 61747 64797 70099 70310 78128 79052 81104 81253 87607 88791 90640 91462 92107 96001 97569 98487 100447 106278 107445 111550 141147 141207 155918 163518 198047 237709 239794 259091],[1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
cbc22468fac4860040971cbe031ea19f8227f966d3a27884a9466a08b869d926f8ce43bf9ba88ee4691958b5a30794f0039b68fdc45f99d3dfc515727c2118d7,0,3183.96,0.522336,0.0760335,2247.62,0.569637,0.0630055,0,85,0,262144,[ 0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 31 32 33 34 38 39 40 41 42 43 44 45 46 47 48 49 50 51 52 53 54 59 60 64 74],[3. 2. 2. 2. 2. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.],[ 0 1 2 3 4 5 6 7 8 9 36 45 46 47 68 69 70 71 72 73 74 75 76 77 78 96 97 98 99 100 101 102 103 104 105 106 112 113 114 302 303 304 928 944 948 952 1624 1625 1626 1657 2654 16970 20440 23125 23858 24125 24348 33711 34189 34569 34699 35707 35765 36001 37044 37939 38432 39438 39543 41952 42951 43753 44569 44846 45642 103813 104899 174276 189763 191089 219811 223729],[1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
647a1a86d4a231f6ea53f9d03cc4065768bac9e94533264d52a474613af4af4f6efa50f598c09acf943be604b22714ee7b5d6cfe5e8d05a2044a5650ca247c07,0,5527.58,0.177085,0.0803395,5264.12,0.116322,0.0787753,0,85,0,262144,[ 0 1 2 3 


prediction


predict,p0,p1
0,1,1e-19
0,1,1e-19
0,1,1e-19
0,1,1e-19
0,1,1e-19
0,1,1e-19
0,1,1e-19
0,1,1e-19
0,1,1e-19
0,1,3.25879e-18


In [19]:
#preds = aml.predict(test)
# or:
preds = aml.leader.predict(test)
print("test")
print(test.tail(10))
print("prediction")
print(preds.tail(10))
# If you need to generate predictions on a test set, you can make
# predictions directly on the `"H2OAutoML"` object, or on the leader
# model object directly

#preds = aml.predict(test)
# or:
preds_over_all_hf = aml.leader.predict(train)
path_out_file1="product_model_prediction/file_prediction_"+process_date+".csv"
output_pred_file=h2o.export_file(frame=preds_over_all_hf, path=path_out_file1, force=False)
#
train_over_all_hf = train
path_out_file2="product_model_prediction/file_train_"+process_date+".csv"
output_pred_file=h2o.export_file(frame=train_over_all_hf, path=path_out_file2, force=False)
#

gbm prediction progress: |████████████████████████████████████████████████| 100%


/opt/cloudera/parcels/Anaconda-5.1.0.1/lib/python3.6/site-packages/h2o/job.py:69: UserWarning: Test/Validation dataset column 'features85_list_indices' has levels not trained on: [[ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23
 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47
 48 49 50 51 52 53 54 55 56 57 58 61 62 63 65 66 79 80 83 84], [ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23
 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47
 48 49 50 51 52 53 54 55 56 58 61 62 63 65 79 80 83 84], [ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23
 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47
 48 49 50 51 52 53 54 56 60 61 64 66 67 75 77], [ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23
 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47
 48 49 50 51 52 54 55 56 57 58 61 62 63 65 66 79 80], [ 0  1  2  3  

test


hash_message,fraud_label,kl_fraud_words,ks_fraud_words,entropy_fraud_words,kl_notfraud_words,ks_notfraud_words,entropy_notfraud_words,features85_type,features85_size,ngramscounts7_type,ngramscounts7_size,features85_list_indices,features85_list_values,ngramscounts7_list_indices,ngramscounts7_list_values
0c983268c70fd7c560320a3a925c414fd448cfad5dbc1287f1f1234aae6b706bdc5ac0c1a43d67efea4f8d4d1c47a4795449ad8284a1d4eef377f2e407ecca15,0,3787.34,0.399077,0.0795858,2450.78,0.722469,0.0547815,0,85,0,262144,[ 0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 31 32 33 34 38 39 40 41 42 43 44 45 46 47 48 49 50 51 52 53 54 55 57 58 59 60 64 74],[3. 2. 2. 2. 2. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 2. 1. 1. 1. 1. 1. 1.],[ 0 1 2 3 4 5 6 7 8 9 23 24 25 26 27 36 45 46 47 65 66 67 68 69 70 71 72 73 74 75 76 77 78 96 97 98 99 100 101 102 103 104 105 106 112 113 114 152 153 154 1455 2641 12207 36645 37149 38512 42844 43290 45175 45622 47181 47315 47754 51188 51421 53447 54992 55174 57563 58819 58824 58831 59045 59285 59421 60606 61074 64791 65510 66339 67109 67692 126259 227408 246021],[1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
751b0d4b88a1ad630b258c03596bf591c7d7d671fe8a54fc802599b80695ca85373e5b918307956930479bfcc5640b2fa91ccadf77c8afcf75dbbeef8d627689,0,3136.01,0.522336,0.0757026,2208.72,0.316835,0.0628293,0,85,0,262144,[ 0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 31 32 33 34 38 39 40 41 42 43 44 45 46 47 48 49 50 51 52 53 54 57 59 60 64 74],[3. 2. 2. 2. 2. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.],[ 0 1 2 3 4 5 6 7 8 9 36 45 46 47 68 69 70 71 72 73 74 75 76 77 78 96 97 98 99 100 101 102 103 104 105 106 112 113 114 1070 1101 1103 1116 1172 1587 1853 1864 1866 7423 7552 7969 23657 46798 59908 60463 61747 64797 70099 70310 78128 79052 81104 81253 87607 88791 90640 91462 92107 96001 97569 98487 100447 106278 107445 111550 141147 141207 155918 163518 198047 237709 239794 259091],[1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
cbc22468fac4860040971cbe031ea19f8227f966d3a27884a9466a08b869d926f8ce43bf9ba88ee4691958b5a30794f0039b68fdc45f99d3dfc515727c2118d7,0,3183.96,0.522336,0.0760335,2247.62,0.569637,0.0630055,0,85,0,262144,[ 0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 31 32 33 34 38 39 40 41 42 43 44 45 46 47 48 49 50 51 52 53 54 59 60 64 74],[3. 2. 2. 2. 2. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.],[ 0 1 2 3 4 5 6 7 8 9 36 45 46 47 68 69 70 71 72 73 74 75 76 77 78 96 97 98 99 100 101 102 103 104 105 106 112 113 114 302 303 304 928 944 948 952 1624 1625 1626 1657 2654 16970 20440 23125 23858 24125 24348 33711 34189 34569 34699 35707 35765 36001 37044 37939 38432 39438 39543 41952 42951 43753 44569 44846 45642 103813 104899 174276 189763 191089 219811 223729],[1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
647a1a86d4a231f6ea53f9d03cc4065768bac9e94533264d52a474613af4af4f6efa50f598c09acf943be604b22714ee7b5d6cfe5e8d05a2044a5650ca247c07,0,5527.58,0.177085,0.0803395,5264.12,0.116322,0.0787753,0,85,0,262144,[ 0 1 2 3 


prediction


predict,p0,p1
0,1,1e-19
0,1,1e-19
0,1,1e-19
0,1,1e-19
0,1,1e-19
0,1,1e-19
0,1,1e-19
0,1,1e-19
0,1,1e-19
0,1,3.25879e-18



gbm prediction progress: |████████████████████████████████████████████████| 100%
Export File progress: |███████████████████████████████████████████████████| 100%
Export File progress: |███████████████████████████████████████████████████| 100%


In [22]:
#
import findspark
findspark.init()
#
import pyspark
from pyspark.sql import functions as pfunc
from pyspark.sql import SQLContext
from pyspark.sql import Window, types
#
import re
import pandas as pd
import numpy as np
from pandas import DataFrame
from pyspark.sql.types import IntegerType
from pyspark.sql.types import FloatType
from pyspark.sql.functions import udf
from pyspark.sql.functions import *
from scipy.stats import kstest
from scipy import stats
#
from pyspark.ml.feature import Tokenizer
from pyspark.ml.feature import RegexTokenizer
#
#import org.apache.spark.ml.feature.NGram
from pyspark.ml.feature import NGram
#
from collections import Counter
#
from pyspark.ml.feature import NGram
#
from pyspark.ml.feature import NGram, CountVectorizer, VectorAssembler
from pyspark.ml import Pipeline
#
from pyspark.mllib.linalg import SparseVector, DenseVector
#
from pyspark.ml.feature import PCA
from pyspark.ml.linalg import Vectors
#
# monotonically_increasing_id allows to use join to merge save size DF
# https://forums.databricks.com/questions/8180/how-to-merge-two-data-frames-column-wise-in-apache.html
from pyspark.sql.functions import monotonically_increasing_id
#
#  FILTER with PySpark SQL Functions F.
from pyspark.sql.types import StringType
from pyspark.sql.functions import udf

def match_func(predict, fraud_label):
    if predict != fraud_label:
        return "no_match"
    else:
        return "match"
match_udf = udf(match_func, StringType())    
#match_udf = udf(lambda (prediction,fraud_label): "no_match" if prediction!=fraud_label else "match", StringType())

#
# Arguments
#
import argparse
## Parse date_of execution
#parser = argparse.ArgumentParser()
#parser.add_argument("--datev1", help="Execution Date")
#args = parser.parse_args()
#if args.datev1:
#    processdate = args.datev1
# GENERAL PREPARATION SCRIPT
#
#  Date in format YYYYMMDD
#process_date = processdate
#if not process_date:
#    process_date = "20181231"
#
process_date = "20190206"
#
sc = pyspark.SparkContext(appName="FraudCanada-Merge-Scoring-Data-From-Model")
sqlContext = SQLContext(sc)
#
input_file_prediction_df="hdfs:///data/staged/ott_dazn/advanced-model-data/scoring-fraud-notfraud-kl-ks-entropy-ngrams7-features-85//dt="+process_date+"/file_prediction_"+process_date+".csv"
#
input_file_train_df="hdfs:///data/staged/ott_dazn/advanced-model-data/scoring-fraud-notfraud-kl-ks-entropy-ngrams7-features-85//dt="+process_date+"/file_train_"+process_date+".csv"
#
output_file1="hdfs:///data/staged/ott_dazn/advanced-model-data/scoring-fraud-notfraud-kl-ks-entropy-ngrams7-features-85//dt="+process_date+"/merged_prediction_train"
#
prediction_df=sqlContext.read.csv(input_file_prediction_df,header=True,inferSchema=True)
prediction_df = prediction_df.withColumn("id1", monotonically_increasing_id())
prediction_df.printSchema()
#
train_df=sqlContext.read.csv(input_file_train_df,header=True,inferSchema=True,multiLine=True)
train_df = train_df.withColumn("id1", monotonically_increasing_id())
train_df.printSchema()
#
merged_prediction_train_df3 = prediction_df.join(train_df, "id1", "outer") #.drop("id1")
#
merged_prediction_train_df3\
.withColumn("match_no_match",match_udf(col('predict').cast('int'),col('fraud_label').cast('int')))\
.sort(merged_prediction_train_df3.id1.desc()).coalesce(1).write.csv(output_file1,header=True)
#
sc.stop()
#

root
 |-- predict: integer (nullable = true)
 |-- p0: double (nullable = true)
 |-- p1: double (nullable = true)
 |-- id1: long (nullable = false)

root
 |-- hash_message: string (nullable = true)
 |-- fraud_label: integer (nullable = true)
 |-- kl_fraud_words: double (nullable = true)
 |-- ks_fraud_words: double (nullable = true)
 |-- entropy_fraud_words: double (nullable = true)
 |-- kl_notfraud_words: double (nullable = true)
 |-- ks_notfraud_words: double (nullable = true)
 |-- entropy_notfraud_words: double (nullable = true)
 |-- features85_type: integer (nullable = true)
 |-- features85_size: integer (nullable = true)
 |-- ngramscounts7_type: integer (nullable = true)
 |-- ngramscounts7_size: integer (nullable = true)
 |-- features85_list_indices: string (nullable = true)
 |-- features85_list_values: string (nullable = true)
 |-- ngramscounts7_list_indices: string (nullable = true)
 |-- ngramscounts7_list_values: string (nullable = true)
 |-- id1: long (nullable = false)



In [21]:
sc.stop()

In [26]:
#
#
import findspark
findspark.init()
#
import pyspark
from pyspark.sql import functions as pfunc
from pyspark.sql import SQLContext
from pyspark.sql import Window, types
#
import re
import pandas as pd
import numpy as np
from pandas import DataFrame
from pyspark.sql.types import IntegerType
from pyspark.sql.types import FloatType
from pyspark.sql.functions import udf
from pyspark.sql.functions import *
from scipy.stats import kstest
from scipy import stats
#
from pyspark.ml.feature import Tokenizer
from pyspark.ml.feature import RegexTokenizer
#
#import org.apache.spark.ml.feature.NGram
from pyspark.ml.feature import NGram
#
from collections import Counter
#
from pyspark.ml.feature import NGram
#
from pyspark.ml.feature import NGram, CountVectorizer, VectorAssembler
from pyspark.ml import Pipeline
#
from pyspark.mllib.linalg import SparseVector, DenseVector
#
from pyspark.ml.feature import PCA
from pyspark.ml.linalg import Vectors
#
#  FILTER with PySpark SQL Functions F.
from pyspark.sql import functions as F
# FUNCTIONS
#
# Arguments
#
import argparse
## Parse date_of execution
#parser = argparse.ArgumentParser()
#parser.add_argument("--datev1", help="Execution Date")
#args = parser.parse_args()
#if args.datev1:
#    processdate = args.datev1
# GENERAL PREPARATION SCRIPT
#
#  Date in format YYYYMMDD
#process_date = processdate
#if not process_date:
#    process_date = "20190113"
#
process_date = "20190206"
#
sc = pyspark.SparkContext(appName="FraudCanada-Search-Suspitious-hashmessage-matching_words-By-AUTOML-NGrams-CountVectorizer-KL-KS-Entropy")
sqlContext = SQLContext(sc)
#
input_fraud="hdfs:///data/staged/ott_dazn/advanced-model-data/fraud-notfraud-canada-tokenizedwords-ngrams-7-features-85/dt="+process_date
#
input_merged_prediction_train="hdfs:///data/staged/ott_dazn/advanced-model-data/scoring-fraud-notfraud-kl-ks-entropy-ngrams7-features-85/dt="+process_date+"/merged_prediction_train/merge_prediction_train_"+process_date+".csv"
#
output_no_match="hdfs:///data/staged/ott_dazn/advanced-model-data/scoring-fraud-notfraud-kl-ks-entropy-ngrams7-features-85/dt="+process_date+"/no_match_hash_message"
#
# The most Frequent would the ones with the max frequency of NGrams85 tokens
pd.options.display.max_colwidth = 512
#
#
# Select Tokens/words form the max frequency of NGrams85 tokens hash_message
notfraud_df=sqlContext.read.json(input_fraud)\
.select(col('hash_message'),col('words'))
notfraud_df.printSchema()
#
merged_df=sqlContext.read.csv(input_merged_prediction_train,header=True,inferSchema=True,multiLine=True)\
.filter(" match_no_match = 'no_match' ")
merged_df.printSchema()
#
join_results_df=merged_df.join(notfraud_df, "hash_message" )\
.withColumn('words_conc',F.concat_ws('',col('words')).cast('string')).drop('words')\
.persist(pyspark.StorageLevel.MEMORY_AND_DISK_2)
join_results_df.printSchema()
#
join_results_df.coalesce(1).write.csv(output_no_match,header=True)
#
### Argument from Merge_train_predict_YYYYMMDD.csv
#        #47bade7fcc6883b26ff81f0681ff8b824d97718c1f9d4584c66f2d58b925f4da9447f6215c62293565820492f7499efea8aed25ef1bf5ddeb36b3872d4ad243d
#var_hash_message='47bade7fcc6883b26ff81f0681ff8b824d97718c1f9d4584c66f2d58b925f4da9447f6215c62293565820492f7499efea8aed25ef1bf5ddeb36b3872d4ad243d'
# Select Tokens/words form the max frequency of NGrams85 tokens hash_message
#standard_notfraud_ngram_words=sqlContext.read.json(input_fraud)
#standard_notfraud_ngram_words.printSchema()
#
#standard_notfraud_words_search=standard_notfraud_ngram_words\
#.withColumn('fraud_search_hash',lit(var_hash_message).cast('string'))\
#.filter(" hash_message=fraud_search_hash ")
#standard_notfraud_words_search.printSchema()
#
#standard_notfraud_words=standard_notfraud_words_search\
#.withColumn('words_conc',F.concat_ws('',col('words')).cast('string'))\
#.select(col('words_conc')).limit(1).toPandas()['words_conc'][0] 
#
#print("From Identifyed Label=0, prediction p=1 : var_hash_message=")
#print(var_hash_message)
#print("Potential fraud value FROM : standard_notfraud_words=")
#print(standard_notfraud_words)
#
sc.stop()

root
 |-- hash_message: string (nullable = true)
 |-- words: array (nullable = true)
 |    |-- element: string (containsNull = true)

root
 |-- id1: integer (nullable = true)
 |-- predict: integer (nullable = true)
 |-- p0: double (nullable = true)
 |-- p1: double (nullable = true)
 |-- hash_message: string (nullable = true)
 |-- fraud_label: integer (nullable = true)
 |-- kl_fraud_words: double (nullable = true)
 |-- ks_fraud_words: double (nullable = true)
 |-- entropy_fraud_words: double (nullable = true)
 |-- kl_notfraud_words: double (nullable = true)
 |-- ks_notfraud_words: double (nullable = true)
 |-- entropy_notfraud_words: double (nullable = true)
 |-- features85_type: integer (nullable = true)
 |-- features85_size: integer (nullable = true)
 |-- ngramscounts7_type: integer (nullable = true)
 |-- ngramscounts7_size: integer (nullable = true)
 |-- features85_list_indices: string (nullable = true)
 |-- features85_list_values: string (nullable = true)
 |-- ngramscounts7_list_ind

In [25]:
sc.stop()